In [ ]:
## Importing the required libraries 

import os
import pandas as pd
import numpy as np
import statsmodels.api as sm 

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE
# Set pandas display options to show more rows and columns to show all the records as needed
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## Reading the input data 
df=pd.read_csv('telecom_churn_data.csv')
df.info()

## data has around 230 columna and 100K rows. 

### Understanding the input Data

In [ ]:
df.shape

In [ ]:
## Checking for the NULL values in the columns
df.isna().sum().any()

## There are NULL values present in the columns, lets check which all columns have the missing values

In [ ]:
print(df.isna().sum()/len(inp_fl)*100)

## Most of the columns have less that 10% of the NULL values present in it, so it should be good enough to use those

In [ ]:
### Check if we have unique customers 
df.mobile_number.nunique()

In [ ]:
df.head()

In [ ]:
## Columns of importance to tag as churn 
##  total_ic_mou_9, total_og_mou_9, vol_2g_mb_9, vol_3g_mb_9

churn_check=df[['mobile_number','total_ic_mou_9', 'total_og_mou_9', 'vol_2g_mb_9', 'vol_3g_mb_9']]
churn_check.info()

In [ ]:
churn_check.head()

In [ ]:
churn_check['churn']=np.where((churn_check['total_ic_mou_9']==0) &
                              (churn_check['total_og_mou_9']==0) &
                              (churn_check['vol_2g_mb_9']==0) &
                              (churn_check['vol_3g_mb_9']==0), 1,0)
churn_check.head()

In [ ]:
(churn_check['churn'].value_counts()/len(churn_check)*100)

## As per the logic to tag churners based on the 9th month's behaviour, i.e. no call and no usage
## only around 10% of customers are there who have churned which makes this problem highly imbalanced in 
## nature

In [ ]:
inp_tag=pd.merge(inp_fl,churn_check[['mobile_number','churn']],on='mobile_number',how='inner')
inp_tag.head()

### Building the data set for EDA and Modelling 

In [ ]:
## we have to split the data based on columns , such that churn behaviour is trained on columns ,6,7,8 and 9th is 
## used for the prediction purpose
all_columns=inp_tag.columns
all_columns

In [ ]:
month9_cols=[col for col in all_columns if '_9' in col]

In [ ]:
## rest of the columns
rest_columns=[col for col in all_columns if col not in month9_cols]

In [ ]:
## base data preparation
base_df=inp_tag[rest_columns]
base_df.head()

In [ ]:
## Check if customers belonging to particular circles 
base_df.circle_id.value_counts()

#### Now understanding Usage, recharge, functionality etc, which will describe how the customer behaviour changes

In [ ]:
## Understanding the recharge schedule and making features of the recharge. 
## Assumption is if recharge amount is decreasing or diminishing then that can be a signal towards churning

recharge_cols=[col for col in rest_columns if 'rech' in col]
recharge_cols

In [ ]:
recharge_df=base_df[['mobile_number']+recharge_cols+['churn']]
recharge_df.tail()

In [ ]:
## We can keep overall recharge as one and data recharge as just a boost to base recharge 
recharge_df['total_rech_data_6'].fillna(0,inplace=True)
recharge_df['total_rech_data_7'].fillna(0,inplace=True)
recharge_df['total_rech_data_8'].fillna(0,inplace=True)

recharge_df['av_rech_amt_data_6'].fillna(0,inplace=True)
recharge_df['av_rech_amt_data_7'].fillna(0,inplace=True)
recharge_df['av_rech_amt_data_8'].fillna(0,inplace=True)


In [ ]:
recharge_df.tail()

In [ ]:
## Only using recharge and data information to understand the behaviour
## drop columns with NAs in it
recharge_df.info()

In [ ]:
## Instead of Dates we can change it to days since the last recharge and average days inbetween the recharges. 
recharge_df[recharge_df['date_of_last_rech_6'].isna()].head()

## It can be seen that people either entered into the system after 6th month and maybe they were there in the system 
## but left the system before the 9th month (assuming both 7th and 8th months has no recharge in it

In [ ]:
## check when it has both 3g and 2g recharges 
recharge_df['recharge_type_data_6_both']=np.where((recharge_df['count_rech_2g_6']>0) & 
                                                  (recharge_df['count_rech_3g_6']>0),1,0)
recharge_df['recharge_type_data_7_both']=np.where((recharge_df['count_rech_2g_7']>0) & 
                                                  (recharge_df['count_rech_3g_7']>0),1,0)
recharge_df['recharge_type_data_8_both']=np.where((recharge_df['count_rech_2g_8']>0) & 
                                                  (recharge_df['count_rech_3g_8']>0),1,0)

In [ ]:
recharge_df.head()

In [ ]:
## Date transformation 
# Custom function to calculate count based on NA values for the subset of columns
# Assuming you have a DataFrame called 'df' with three columns 'col1', 'col2', and 'col3'
subset_columns = ['date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8']
def count_based_on_na(row):
    num_nas = row[subset_columns].isnull().sum()
    if num_nas == 0:
        return 3
    elif num_nas == 1:
        return 2
    elif num_nas == 2:
        return 1
    else:
        return 0

# Apply the custom function row-wise to calculate the count for each row in the subset of columns

recharge_df['rech_month_active']=recharge_df.apply(count_based_on_na, axis=1)


In [ ]:
## Change recharge dates to 0 and 1 , as categorical 
recharge_df['date_of_last_rech_6']= np.where(recharge_df['date_of_last_rech_6'].isna(),0,1)
recharge_df['date_of_last_rech_7']=np.where(recharge_df['date_of_last_rech_7'].isna(),0,1)
recharge_df['date_of_last_rech_8']=np.where(recharge_df['date_of_last_rech_8'].isna(),0,1)


In [ ]:
recharge_df.head()
# Drop columns with NA (missing values)
recharge_df = recharge_df.dropna(axis=1)
recharge_df.head()

## Idea is , if mobile is getting recharged and it has any data pack 3g or 2g as recharged in it, then the 
## recharge count of data and its amounts will impact, either of the data type will not impact, but person
## recharging both data packs might have different behaviour

In [ ]:
## getting slope and change in recharge amounts, to understand if that impact of M-O-M change has on churn
recharge_df['rech_amt_7n6_slope']=recharge_df['total_rech_amt_7']-recharge_df['total_rech_amt_6']
recharge_df['rech_amt_8n7_slope']=recharge_df['total_rech_amt_8']-recharge_df['total_rech_amt_7']
recharge_df['rech_slope_tag']=np.where((recharge_df['rech_amt_7n6_slope']<=0)&
                                       (recharge_df['rech_amt_8n7_slope']<=0),-1,
                               np.where((recharge_df['rech_amt_7n6_slope']>0)&
                                       (recharge_df['rech_amt_8n7_slope']>0),1,0))
                                   

In [ ]:
recharge_df['avg_rech_amt']=(recharge_df['total_rech_amt_6']+recharge_df['total_rech_amt_7']+recharge_df['total_rech_amt_8'])/3
recharge_df.head() 

## recharge data is completed, with average recharge and the kind of data recharge

In [ ]:
base_df.isna().sum()/len(base_df)*100

In [ ]:
# Function to calculate the percentage of null values in each column
def calculate_null_percentage(column):
    total_rows = len(base_df)
    null_count = column.isnull().sum()
    null_percentage = (null_count / total_rows) * 100
    return null_percentage

# Calculate the threshold percentage (e.g., 30%)
threshold_percentage = 50

# Drop columns with null values greater than the threshold percentage of 50%
base_df_filtered = base_df.drop(base_df.columns[base_df.apply(calculate_null_percentage) > threshold_percentage], axis=1)


In [ ]:
base_df_filtered.head()

In [ ]:
# Dictionary to store the column names and their unique values
columns_with_unique_values = {}

# Loop through each column
for column in base_df_filtered.columns:
    unique_values = base_df_filtered[column].unique()
    num_unique_values = base_df_filtered[column].nunique()
    
    # Check if the number of unique values is less than 10
    if num_unique_values < 5:
        columns_with_unique_values[column] = {'Num_Unique_Values': num_unique_values, 'Unique_Values': unique_values}


In [ ]:
columns_with_unique_values
## It can be seen that these columns does not have much information in it, so we can remove these columns too

In [ ]:
columns_remove=list(columns_with_unique_values.keys())
columns_remove=columns_remove[:len(columns_remove)-1]
columns_remove

In [ ]:
base_df_filtered.drop(columns=columns_remove,inplace=True)
base_df_filtered.shape

## Now we have 124 columns, we can remove recharge ones from this too 

In [ ]:
base_df_filtered=base_df_filtered[[col for col in base_df_filtered.columns if col not in recharge_cols]]
base_df_filtered.shape

In [ ]:
base_df_filtered.head(20)

In [ ]:
#base_df_filtered.fillna(0,inplace=True)

In [ ]:
## Onnet VS Offnet usage, -if user has more out of network calls or usage , does that impact the churn rate 
## Given this T2T, T2M, T2o etc will be covered as part of it so we can remove all those columns and just focust on 
## onnet vs offnet 


In [ ]:
def get_highly_correlated_columns(df, threshold=0.8):
    """
    Function to find column pairs with correlation greater than a specified threshold.

    Parameters:
        df (pandas.DataFrame): The DataFrame containing the data.
        threshold (float, optional): The correlation threshold. Defaults to 0.8.

    Returns:
        pandas.DataFrame: DataFrame containing correlated column pairs and their correlation values.
    """
    # Compute the correlation matrix
    correlation_matrix = df.corr()

    # Find pairs of columns with correlation greater than the threshold
    correlated_column_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) > threshold:
                col_pair = (correlation_matrix.columns[i], correlation_matrix.columns[j], correlation_matrix.iloc[i, j])
                correlated_column_pairs.append(col_pair)

    # Create a DataFrame to store the correlated column pairs and their correlation values
    df_correlated = pd.DataFrame(correlated_column_pairs, columns=['Column1', 'Column2', 'Correlation'])

    return df_correlated



# Assuming you have a DataFrame called 'df'
correlated_columns = get_highly_correlated_columns(base_df_filtered, threshold=0.75)
print(correlated_columns)


In [ ]:
base_df_filtered.shape

In [ ]:
## We can remove the columns of sachet and rch. as recharge columns we already have 
drop_cols2=['last_day_rch_amt_6',
       'last_day_rch_amt_7', 'last_day_rch_amt_8']
base_df_filtered.drop(columns=drop_cols2,inplace=True)

In [ ]:
base_df_filtered.tail(10)

#### How Churn is impacted by multiple scenarios

In [ ]:
recharge_df.drop(columns='churn',inplace=True)

base_data_all=pd.merge(base_df_filtered,recharge_df,on='mobile_number',how='inner')

In [ ]:
base_data_all.head()

### Filtering 70th percentile

In [ ]:
## Check for missing value and treatment in the high value customers, lets create the dataset first
#Deriving Average recharge amount of June and July.
base_data_all['average_rech_amt_6n7']=(base_data_all['total_rech_amt_6']+base_data_all['total_rech_amt_7'])/2

#Filtering based HIGH VALUED CUSTOMERS based on (Average_rech_amt_6n7 >= 70th percentile of Average_rech_amt_6n7)
base_data_high_val=base_data_all[(base_data_all['average_rech_amt_6n7']>= base_data_all['average_rech_amt_6n7'].quantile(0.7))]



In [ ]:
base_data_high_val.shape

In [ ]:
## Check missing value by each month columns 
base_data_high_val.isna().sum()

In [ ]:
high_val_data_cols=list(base_data_high_val.columns)

In [ ]:
## Missing values in 6th months
month6_data=base_data_high_val[[col for col in high_val_data_cols if '_6' in col]]
month6_data.isna().sum()/len(month6_data)*100

## it can be seen that , all the MOU columns have a similar % of missing value in it, which can be related to 
## some pattern with the users, as this cannot be random. If they are not using calling plans then their total og
## and ic should also be zero, which we can confirm 

In [ ]:
month6_data[month6_data['onnet_mou_6'].isna()]['total_ic_mou_6'].unique()
## Their total mou is also zero, which means they are not using any call packs, we can fill these NA values with 0

In [ ]:
base_data_high_val[list(month6_data.columns)]=base_data_high_val[list(month6_data.columns)].fillna(0)

In [ ]:
## Now similarly we can check for month 7 
month7_data=base_data_high_val[[col for col in high_val_data_cols if '_7' in col]]
month7_data.isna().sum()/len(month6_data)*100
## It has similar chance, we can fill it again with 0 

In [ ]:
base_data_high_val[list(month7_data.columns)]=base_data_high_val[list(month7_data.columns)].fillna(0)

In [ ]:
## Now similarly we can check for month 8
month8_data=base_data_high_val[[col for col in high_val_data_cols if '_8' in col]]
month8_data.isna().sum()/len(month8_data)*100

In [ ]:
base_data_high_val[list(month8_data.columns)]=base_data_high_val[list(month8_data.columns)].fillna(0)

In [ ]:
base_data_high_val.isna().any()
## Now there are no missing values in this data

In [ ]:
## Checking churn with change average revenue per use 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

arpu_data=base_data_high_val.groupby(['churn'])['arpu_6','arpu_7','arpu_8'].agg(np.median).reset_index()
arpu_data_melt = arpu_data.melt(id_vars='churn',value_vars=['arpu_6','arpu_7','arpu_8'], var_name='Columns', value_name='Values')

sns.barplot(x='Columns', y='Values',hue='churn', data=arpu_data_melt)
plt.title('Median Revenue by Churn')
plt.show()

## People having higher chance of churn can see a dip in ARPU which is significant enough from, we can now get derived 
## metrics to mark the slope by users
base_data_high_val['arpu_slope_7n6']=base_data_high_val['arpu_7']-base_data_high_val['arpu_6']
base_data_high_val['arpu_slope_8n7']=base_data_high_val['arpu_8']-base_data_high_val['arpu_7']
base_data_high_val['arpu_slope_tag']=np.where((base_data_high_val['arpu_slope_7n6']<=0)&
                                       (base_data_high_val['arpu_slope_8n7']<=0),-1,
                               np.where((base_data_high_val['arpu_slope_7n6']>0)&
                                       (base_data_high_val['arpu_slope_8n7']>0),1,0))


In [ ]:
## Checking onnet vs offnet against the churn
## if user has higher offnet 
arpu_data=base_data_high_val.groupby(['churn'])['onnet_mou_6','onnet_mou_7','onnet_mou_8','offnet_mou_6','offnet_mou_7','offnet_mou_8'].agg(np.mean).reset_index()
arpu_data_melt = arpu_data.melt(id_vars='churn',value_vars=['onnet_mou_6','onnet_mou_7','onnet_mou_8',
                                                           'offnet_mou_6','offnet_mou_7','offnet_mou_8'], var_name='Columns', value_name='Values')

sns.barplot(x='Columns', y='Values',hue='churn', data=arpu_data_melt)
plt.title('Network type by Churn')
plt.show()

# ## People having higher chance of churn by reduction in onnet as well as offnet
## let take the ratio of offnet by onnet if there is usage distirbution impacting it 
base_data_high_val['on_off_ratio_6']=np.where((base_data_high_val['onnet_mou_6']==0) | 
                                         (base_data_high_val['offnet_mou_6']==0),0,
                                          base_data_high_val['onnet_mou_6']/(base_data_high_val['onnet_mou_6']+base_data_high_val['onnet_mou_6']))

base_data_high_val['on_off_ratio_7']=np.where((base_data_high_val['onnet_mou_7']==0) | 
                                         (base_data_high_val['offnet_mou_7']==0),0,
                                          base_data_high_val['onnet_mou_7']/(base_data_high_val['onnet_mou_7']+base_data_high_val['onnet_mou_7']))

base_data_high_val['on_off_ratio_8']=np.where((base_data_high_val['onnet_mou_8']==0) | 
                                         (base_data_high_val['offnet_mou_8']==0),0,
                                          base_data_high_val['onnet_mou_8']/(base_data_high_val['onnet_mou_8']+base_data_high_val['onnet_mou_8']))


arpu_data=base_data_high_val.groupby(['churn'])['on_off_ratio_6','on_off_ratio_7','on_off_ratio_8'].agg(np.mean).reset_index()
arpu_data_melt = arpu_data.melt(id_vars='churn',value_vars=['on_off_ratio_6','on_off_ratio_7','on_off_ratio_8'],
                                var_name='Columns', value_name='Values')

sns.barplot(x='Columns', y='Values',hue='churn', data=arpu_data_melt)
plt.title('Onnet-Offnet ratio by Churn')
plt.show()


## However if we see from a onnet to offnet ratio, the ones churning are basically due to decrease in onnet usage
## so if the one reducing the local usage has higher chance to churn. People not churning has around 45/55 split of 
## onnet vs offnet usage

In [ ]:
base_data_high_val.head()

In [ ]:

base_data_high_val['onnet_offnet_slope'] = (base_data_high_val['on_off_ratio_8']-base_data_high_val['on_off_ratio_6']).add(base_data_high_val['on_off_ratio_7']).div(2)
base_data_high_val.head()

In [ ]:
## Understand how we are getting impact by high revenue customer and low value customer segement 
## High valuable customers can be described by arpu and aon of the customer 
## We can create divide the customers by deciles to understand how much percentage of customers are getting 
## churned in which deciles 

## Deciling by network age time and ARPU 

## Understanding the time concept 
# Create a histogram with hue using displot
plt.figure(figsize=(10,20))
sns.displot(data=base_data_high_val, x='aon', hue='churn', kde=True, multiple='stack', palette='colorblind')
plt.title('Churn distribution by customer time')
## Both churn vs non churn customers have skewed distribution, but it can be seen that people spedning more than 
## 1000 days with the operator has lesser chance to churn as most of the churn is happening with the new customers



In [ ]:
### 
base_data_high_val[base_data_high_val['arpu_6']<0]

In [ ]:
## We can combine roaming as one , idea is if person uses either IC or OG roaming in any month then it has roaming
## active for that month, also the roaming minutes will directly impact the churn too, hypothesis is 
## if my network is benificial with roaming then high MOU customer will stick the network
all_columns=list(base_data_high_val.columns)

roam_cols=[col for col in all_columns if 'roam' in col]
print(roam_cols)

local_cols=[col for col in all_columns if 'loc' in col]
print(local_cols)

special_cols=[col for col in all_columns if 'spl' in col]
print(special_cols)

isd_cols=[col for col in all_columns if 'isd' in col]
print(isd_cols)

std_cols=[col for col in all_columns if 'std' in col]
print(std_cols)

other_cols=[col for col in all_columns if 'other' in col]
print(other_cols)



In [ ]:
local_df=base_data_high_val[['mobile_number']+local_cols+['churn']]

## Checking with overall total local = t2t+t2m+t2f+t2c, this change in % in these behaviours can impact 
## how the churn eventually happens 

## Changing everything to percentage rather than the actual numbers 
for col in local_cols:
    if '6' in col:
        if 'loc_og' in col:
            local_df[col]=round(local_df[col]/local_df['loc_og_mou_6']*100)
        else:
            local_df[col]=round(local_df[col]/local_df['loc_ic_mou_6']*100)
    elif '7' in col:
        if 'loc_og' in col :
            local_df[col]=round(local_df[col]/local_df['loc_og_mou_7']*100)
        else:
            local_df[col]=round(local_df[col]/local_df['loc_ic_mou_7']*100)
    else:
        if 'loc_og' in col:
            local_df[col]=round(local_df[col]/local_df['loc_og_mou_8']*100)
        else:
            local_df[col]=round(local_df[col]/local_df['loc_ic_mou_8']*100)
        
        
            
local_df.fillna(0,inplace=True)
local_df.head()

In [ ]:
local_df.drop(columns=['loc_og_mou_6', 'loc_og_mou_7', 'loc_og_mou_8','loc_ic_mou_6', 'loc_ic_mou_7', 'loc_ic_mou_8'],inplace=True)

In [ ]:
local_df.columns[1:-1]

In [ ]:
## How the churn changes with change in local usage behaviour
local_df_melt=local_df.melt(id_vars=['mobile_number','churn'],value_vars=list(local_df.columns[1:-1]),
                           var_name='Tag',value_name='Values')
local_df_melt['month']=local_df_melt['Tag'].apply(lambda x: x.split('_')[-1])
local_df_melt['ic_og']=local_df_melt['Tag'].apply(lambda x: x.split('_')[1])
local_df_melt['call_type']=local_df_melt['Tag'].apply(lambda x: x.split('_')[2])
local_df_melt.drop(columns='Tag',inplace=True)
local_df_melt.head()

In [ ]:
plt.figure(figsize=(10,10))
local_df_melt.groupby(['ic_og','month','call_type','churn'])['Values'].agg(np.mean).plot(kind='bar')


The ones churning have a significant reduction in t2m in the 8th month, meaning user signiicantly stopped calling to other operators using our network

In [ ]:
arpu_data_melt = arpu_data.melt(id_vars='churn',value_vars=['on_off_ratio_6','on_off_ratio_7','on_off_ratio_8'],
                                var_name='Columns', value_name='Values')

In [ ]:
std_df=base_data_high_val[['mobile_number']+std_cols+['churn']]

## Checking with overall total std = t2t+t2m+t2f+t2c, this change in % in these behaviours can impact 
## how the churn eventually happens 

## Changing everything to percentage rather than the actual numbers 
for col in std_cols:
    if '6' in col:
        if 'std_og' in col:
            std_df[col]=round(std_df[col]/std_df['std_og_mou_6']*100)
        else:
            std_df[col]=round(std_df[col]/std_df['std_ic_mou_6']*100)
    elif '7' in col:
        if 'std_og' in col :
            std_df[col]=round(std_df[col]/std_df['std_og_mou_7']*100)
        else:
            std_df[col]=round(std_df[col]/std_df['std_ic_mou_7']*100)
    else:
        if 'std_og' in col:
            std_df[col]=round(std_df[col]/std_df['std_og_mou_8']*100)
        else:
            std_df[col]=round(std_df[col]/std_df['std_ic_mou_8']*100)
        
        
            
std_df.fillna(0,inplace=True)
std_df.head()

In [ ]:
std_df.drop(columns=['std_og_mou_6', 'std_og_mou_7', 'std_og_mou_8','std_ic_mou_6', 'std_ic_mou_7', 'std_ic_mou_8'],inplace=True)

std_df.columns[1:-1]

## How the churn changes with change in std usage behaviour
std_df_melt=std_df.melt(id_vars=['mobile_number','churn'],value_vars=list(std_df.columns[1:-1]),
                           var_name='Tag',value_name='Values')
std_df_melt['month']=std_df_melt['Tag'].apply(lambda x: x.split('_')[-1])
std_df_melt['ic_og']=std_df_melt['Tag'].apply(lambda x: x.split('_')[1])
std_df_melt['call_type']=std_df_melt['Tag'].apply(lambda x: x.split('_')[2])
std_df_melt.drop(columns='Tag',inplace=True)
std_df_melt.head()

plt.figure(figsize=(10,10))
std_df_melt.groupby(['ic_og','month','call_type','churn'])['Values'].agg(np.mean).plot(kind='bar')

## STD customers calling in the same network in the good phase are churning out more.


In [ ]:
## Checking if roaming and std has high correlation, and we can remove roaming from it 
# Assuming you have a DataFrame called 'df'

roam_std_df=base_data_high_val[['mobile_number']+std_cols+roam_cols+['churn']]
correlated_columns = get_highly_correlated_columns(roam_std_df, threshold=0.7)
print(correlated_columns)

## roaming has no relation with STD, so we can check the impact of Roaming on the churn

In [ ]:
## Checking if roaming and std has 

roam_df=base_data_high_val[['mobile_number']+roam_cols+['churn']]

roam_df['roam_6_total']=roam_df['roam_ic_mou_6']+roam_df['roam_og_mou_6']
roam_df['roam_7_total']=roam_df['roam_ic_mou_7']+roam_df['roam_og_mou_7']
roam_df['roam_8_total']=roam_df['roam_ic_mou_8']+roam_df['roam_og_mou_8']
roam_df['roam_tag']=np.where((roam_df['roam_6_total']==0) & (roam_df['roam_7_total']==0)& (roam_df['roam_8_total']==0),
                            'No Roaming',np.where((roam_df['roam_6_total']>0) | (roam_df['roam_7_total']>0) &
                                                 (roam_df['roam_8_total']==0),'Good Phase Roaming',
                            np.where((roam_df['roam_6_total']==0) & (roam_df['roam_7_total']==0)& (roam_df['roam_8_total']>0),
                                    'Action Phase Roaming', 'All Time Roaming')) )

print('Roaming Active all time')
print(roam_df[roam_df['roam_tag']=='All Time Roaming']['churn'].value_counts()/len(roam_df[roam_df['roam_tag']=='All Time Roaming']))

print('Roaming not active')
print(roam_df[roam_df['roam_tag']=='No Roaming']['churn'].value_counts()/len(roam_df[roam_df['roam_tag']=='No Roaming']))

print('Only Good Phase active')
print(roam_df[roam_df['roam_tag']=='Good Phase Roaming']['churn'].value_counts()/len(roam_df[roam_df['roam_tag']=='Good Phase Roaming']))

print('Only Action Phase Active active')
print(roam_df[roam_df['roam_tag']=='Action Phase Roaming']['churn'].value_counts()/len(roam_df[roam_df['roam_tag']=='Action Phase Roaming']))



## users actively using roaming , even for once in a 3 month time has higher chance to churn against people
## who never used roaming in these 3 months, All time active roamers along with action phase roamers, ~25% of those
## users are leaving the operator, so somewhere roaming plans are not as lucrative as can be seen



In [ ]:
## No checking plans of special uses and its impact on the churn value 
special_df=base_data_high_val[['mobile_number']+special_cols+['churn']]

special_df['special_6_total']=special_df['spl_ic_mou_6']+special_df['spl_og_mou_6']
special_df['special_7_total']=special_df['spl_ic_mou_7']+special_df['spl_og_mou_7']
special_df['special_8_total']=special_df['spl_ic_mou_8']+special_df['spl_og_mou_8']
special_df['special_tag']=np.where((special_df['special_6_total']==0) & (special_df['special_7_total']==0)& (special_df['special_8_total']==0),
                            'No Special Use',np.where((special_df['special_6_total']>0) | (special_df['special_7_total']>0) &
                                                 (special_df['special_8_total']==0),'Good Phase Special Use',
                            np.where((special_df['special_6_total']==0) & (special_df['special_7_total']==0)& (special_df['special_8_total']>0),
                                    'Action Phase Special Use', 'All Time Special Use')) )

print('Special Use Active all time')
print(special_df[special_df['special_tag']=='All Time Special Use']['churn'].value_counts()/len(special_df[special_df['special_tag']=='All Time Special Use']))

print('Special Use not active')
print(special_df[special_df['special_tag']=='No Special Use']['churn'].value_counts()/len(special_df[special_df['special_tag']=='No Special Use']))

print('Only Good Phase active')
print(special_df[special_df['special_tag']=='Good Phase Special Use']['churn'].value_counts()/len(special_df[special_df['special_tag']=='Good Phase Special Use']))

print('Only Action Phase Active')
print(special_df[special_df['special_tag']=='Action Phase Special Use']['churn'].value_counts()/len(special_df[special_df['special_tag']=='Action Phase Special Use']))

## Special usage has better chance of not churning though, especially in the action phase against the ones who
## never used a special pack, we we can see the customer not using a special pack, we can recommend that 
## in their action phase to reduce the churn from 12% to 2% from these figures

In [ ]:
## checking plans of isd uses and its impact on the churn value 
isd_df=base_data_high_val[['mobile_number']+isd_cols+['churn']]

isd_df['isd_6_total']=isd_df['isd_ic_mou_6']+isd_df['isd_og_mou_6']
isd_df['isd_7_total']=isd_df['isd_ic_mou_7']+isd_df['isd_og_mou_7']
isd_df['isd_8_total']=isd_df['isd_ic_mou_8']+isd_df['isd_og_mou_8']
isd_df['isd_tag']=np.where((isd_df['isd_6_total']==0) & (isd_df['isd_7_total']==0)& (isd_df['isd_8_total']==0),
                            'No isd Use',np.where((isd_df['isd_6_total']>0) | (isd_df['isd_7_total']>0) &
                                                 (isd_df['isd_8_total']==0),'Good Phase isd Use',
                            np.where((isd_df['isd_6_total']==0) & (isd_df['isd_7_total']==0)& (isd_df['isd_8_total']>0),
                                    'Action Phase isd Use', 'All Time isd Use')) )

print('isd Use Active all time')
print(isd_df[isd_df['isd_tag']=='All Time isd Use']['churn'].value_counts()/len(isd_df[isd_df['isd_tag']=='All Time isd Use']))

print('isd Use not active')
print(isd_df[isd_df['isd_tag']=='No isd Use']['churn'].value_counts()/len(isd_df[isd_df['isd_tag']=='No isd Use']))

print('Only Good Phase active')
print(isd_df[isd_df['isd_tag']=='Good Phase isd Use']['churn'].value_counts()/len(isd_df[isd_df['isd_tag']=='Good Phase isd Use']))

print('Only Action Phase Active')
print(isd_df[isd_df['isd_tag']=='Action Phase isd Use']['churn'].value_counts()/len(isd_df[isd_df['isd_tag']=='Action Phase isd Use']))


## ISD usage does not significant change in Usage vs Non-usage, so it is not directly changing the churn rate, 
## but still action phase has improve the churn as compared to no-usage at all

In [ ]:

other_df=base_data_high_val[['mobile_number']+other_cols+['churn']]

other_df['other_6_total']=other_df['ic_others_6']+other_df['og_others_6']
other_df['other_7_total']=other_df['ic_others_7']+other_df['og_others_7']
other_df['other_8_total']=other_df['ic_others_8']+other_df['og_others_8']
other_df['other_tag']=np.where((other_df['other_6_total']==0) & (other_df['other_7_total']==0)& (other_df['other_8_total']==0),
                            'No other Use',np.where((other_df['other_6_total']>0) | (other_df['other_7_total']>0) &
                                                 (other_df['other_8_total']==0),'Good Phase other Use',
                            np.where((other_df['other_6_total']==0) & (other_df['other_7_total']==0)& (other_df['other_8_total']>0),
                                    'Action Phase other Use', 'All Time other Use')) )

print('other Use Active all time')
print(other_df[other_df['other_tag']=='All Time other Use']['churn'].value_counts()/len(other_df[other_df['other_tag']=='All Time other Use']))

print('other Use not active')
print(other_df[other_df['other_tag']=='No other Use']['churn'].value_counts()/len(other_df[other_df['other_tag']=='No other Use']))

print('Only Good Phase active')
print(other_df[other_df['other_tag']=='Good Phase other Use']['churn'].value_counts()/len(other_df[other_df['other_tag']=='Good Phase other Use']))

print('Only Action Phase Active')
print(other_df[other_df['other_tag']=='Action Phase other Use']['churn'].value_counts()/len(other_df[other_df['other_tag']=='Action Phase other Use']))


## one interesting to watch here is others and isd service numbers are almost similar , probably they are closely 
## related and ISD might be a part of other services provided, we can check the correlation between them and remove
## if needed

In [ ]:
## correlation between ISD and other services 
others_isd_df=pd.merge(isd_df,other_df,on=['mobile_number','churn'],how='inner')
correlated_columns = get_highly_correlated_columns(others_isd_df, threshold=0.7)
print(correlated_columns)

## ISD is not a related to other services, but still it has a similar impact on the set of users which either used 
## ISD or others 
print('    ')
print('other  & ISD Use Active all time')
print(others_isd_df[(others_isd_df['other_tag']=='All Time other Use') & 
                   (others_isd_df['isd_tag']=='All Time isd Use')]['churn'].value_counts())

## There are only 72 users which have used ISD and other services together, and around 5% of them only churned
## which is small number as compared to other 

In [ ]:
## Checking the impact of data pack usage and its impact on the churn rates
base_data_high_val.head()

In [ ]:
data_2g_cols=[col for col in all_columns if '2g' in col]
data_3g_cols=[col for col in all_columns if '3g' in col]
data_cols=data_2g_cols+data_3g_cols
data_cols=data_cols[:-1]

In [ ]:
## first we have to understand if people use 2g and 3g together 
data_usage_df=base_data_high_val[['mobile_number']+data_cols+['churn']]
data_usage_df.head()

In [ ]:
# Usage 
data_usage_df['delta_vol_2g'] = data_usage_df['vol_2g_mb_8'] - data_usage_df['vol_2g_mb_6'].add(data_usage_df['vol_2g_mb_7']).div(2)
data_usage_df['delta_vol_3g'] = data_usage_df['vol_3g_mb_8'] - data_usage_df['vol_3g_mb_6'].add(data_usage_df['vol_3g_mb_7']).div(2)

In [ ]:
## check the change in data usage by churn 
data_usage_df['data_tag']=np.where((data_usage_df['delta_vol_2g']<0)&(data_usage_df['delta_vol_3g']>0),
                                  '3g_transition',
                           np.where((data_usage_df['delta_vol_2g']>0)&(data_usage_df['delta_vol_3g']<0),
                                   '2g_transition',
                           np.where((data_usage_df['delta_vol_2g']==0)&(data_usage_df['delta_vol_3g']>0),
                                   '3g_user_positive',
                           np.where((data_usage_df['delta_vol_2g']>0)&(data_usage_df['delta_vol_3g']==0),
                                    '2g_user_positive',
                           np.where((data_usage_df['delta_vol_2g']>0)&(data_usage_df['delta_vol_3g']>0),
                                    'data_lover',
                           np.where(((data_usage_df['delta_vol_2g']<0) & (data_usage_df['delta_vol_3g']==0)) |
                                    ((data_usage_df['delta_vol_2g']==0) & (data_usage_df['delta_vol_3g']<0)),
                                    'data_use_reducer','data_non_lover'
                                   ))))))

In [ ]:
data_usage_grouped=data_usage_df.groupby(['data_tag','churn'])['mobile_number'].count().reset_index()
data_usage_grouped['total']=data_usage_grouped.groupby(['data_tag'])['mobile_number'].transform(np.sum)
data_usage_grouped['percent']=data_usage_grouped['mobile_number']/data_usage_grouped['total']
plt.figure(figsize=(10,5))
sns.barplot(x='data_tag',y='percent',data=data_usage_grouped,hue='churn')
plt.show()

## People who do love using data, are not moving out , ones who have not used the data at all are churning

In [ ]:
### Sachet  vs sachet VS vbc usage of data 
data_usage_df['vbc_delta']=data_usage_df['aug_vbc_3g'] - data_usage_df['jun_vbc_3g'].add(data_usage_df['jul_vbc_3g']).div(2)
data_usage_df['sachet_delta_2g']=data_usage_df['sachet_2g_8'] - data_usage_df['sachet_2g_6'].add(data_usage_df['sachet_2g_7']).div(2)
data_usage_df['sachet_delta_3g']=data_usage_df['sachet_3g_8'] - data_usage_df['sachet_3g_8'].add(data_usage_df['sachet_3g_8']).div(2)

data_usage_df['sachet_delta_2g']=data_usage_df['sachet_2g_8'] - data_usage_df['sachet_2g_6'].add(data_usage_df['sachet_2g_7']).div(2)
data_usage_df['sachet_delta_3g']=data_usage_df['sachet_3g_8'] - data_usage_df['sachet_3g_8'].add(data_usage_df['sachet_3g_8']).div(2)



In [ ]:
sns.scatterplot(data=data_usage_df,x='vbc_delta',y='sachet_delta_2g',hue='churn')
plt.title('VBC usage versus 2g sachet recharge')
plt.show()
## Churn and reduction in data usage of 2g sachets and VBC are related

sns.scatterplot(data=data_usage_df,x='vbc_delta',y='sachet_delta_3g',hue='churn')
plt.title('VBC usage versus 3g sachet recharge')
plt.show()
## If a person with no change in 3g sachets but only dependent on VBC has impact on churn, 3g has no significance 
## in the non-churning 


In [ ]:
## making the final analytical data with the derived columns 
local_df.columns=[col+'_perc' if col not in ['mobile_number','churn'] else col for col in list(local_df.columns)]
local_df.head()

In [ ]:
### std df
std_df.columns=[col+'_perc' if col not in ['mobile_number','churn'] else col for col in list(std_df.columns)]
std_df.head()

In [ ]:
local_df.drop(columns='churn',inplace=True)

In [ ]:
std_df.drop(columns='churn',inplace=True)

In [ ]:
roam_df=roam_df[['mobile_number','roam_6_total', 'roam_7_total', 'roam_8_total', 'roam_tag']]

In [ ]:
isd_df=isd_df[['mobile_number','isd_6_total', 'isd_7_total', 'isd_8_total', 'isd_tag']]

In [ ]:
other_df=other_df[['mobile_number','other_6_total', 'other_7_total', 'other_8_total', 'other_tag']]

In [ ]:
special_df=special_df[['mobile_number','special_6_total', 'special_7_total', 'special_8_total', 'special_tag']]

In [ ]:
data_usage_df=data_usage_df[['mobile_number','delta_vol_2g', 'delta_vol_3g', 'data_tag',
       'vbc_delta', 'sachet_delta_2g', 'sachet_delta_3g']]

In [ ]:
from functools import reduce

# Assuming you have multiple DataFrames called df1, df2, df3, etc. that you want to merge

# List of DataFrames to merge
dfs_to_merge = [base_data_high_val, local_df, std_df,roam_df,isd_df,other_df,special_df,data_usage_df]

# Merge the DataFrames using reduce() and merge()
merged_df = reduce(lambda left, right: pd.merge(left, right, on='mobile_number'), dfs_to_merge)

# Display the merged DataFrame
merged_df.head()


In [ ]:
merged_df.drop(columns='sep_vbc_3g',inplace=True)

In [ ]:
merged_df.shape

In [ ]:
merged_df.dtypes

In [ ]:
# Select columns with object data type
object_columns = merged_df.select_dtypes(include='object')
object_columns=object_columns.columns
object_columns

In [ ]:
## Create Dummy variables out of it 
dummy_vars = pd.get_dummies(merged_df[object_columns], drop_first=True, prefix=object_columns, prefix_sep='_')
dummy_vars.head()

In [ ]:
reference_cols = dummy_vars.filter(regex='.*Others$').columns.to_list() # Using category 'Others' in each column as reference. 
dummy_vars.drop(columns=reference_cols, inplace=True)
reference_cols

In [ ]:
# concatenating dummy variables with original 'data'
merged_df.drop(columns=object_columns, inplace=True) # dropping original categorical columns
merged_df = pd.concat([merged_df, dummy_vars], axis=1)
merged_df.head()

### This following section contains

Test Train Split
Class Imbalance
Standardization
- Modelling
  - Model 1 : Logistic Regression with RFE & Manual Elimination ( Interpretable Model )
  - Model 2 : PCA + Logistic Regression
  - Model 3 : PCA + Random Forest Classifier
  

In [ ]:
merged_df.drop(columns=['mobile_number'],inplace=True)
# Replace inf with 0
merged_df = merged_df.replace([np.inf, -np.inf], 0)

In [ ]:
# Looking at quantiles from 0.90 to 1. 
merged_df.quantile(np.arange(0.9,1.01,0.01)).style.bar()


In [ ]:
merged_df.shape

In [ ]:
## Train-Test Split
y = merged_df.pop('churn') # Predicted / Target Variable
X = merged_df # Predictor variables

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7, random_state=42)

In [ ]:
y.value_counts(normalize=True).to_frame()

In [ ]:
# Ratio of classes 
class_0 = y[y == 0].count()
class_1 = y[y == 1].count()

print(f'Class Imbalance Ratio : {round(class_1/class_0,3)}')

## We can use use SMOTE to make these imbalanced classes balanced

In [ ]:
#!pip install imblearn --trusted-host pypi.org --trusted-host files.pythonhosted.org

In [ ]:


from imblearn.over_sampling import SMOTE
smt = SMOTE(random_state=42, k_neighbors=5)

# Resampling Train set to account for class imbalance

X_train_resampled, y_train_resampled= smt.fit_resample(X_train, y_train)
X_train_resampled.head()

In [ ]:
# columns with numerical data
condition1 = merged_df.dtypes == 'int'
condition2 = merged_df.dtypes == 'float'
numerical_vars = merged_df.columns[condition1 | condition2].to_list()
# Standard scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 

# Fit and transform train set 
X_train_resampled[numerical_vars] = scaler.fit_transform(X_train_resampled[numerical_vars])

# Transform test set
X_test[numerical_vars] = scaler.transform(X_test[numerical_vars])


In [ ]:
# summary statistics of standardized variables
round(X_train_resampled.describe(),2)

### Baseline Logistic Regression Model

In [ ]:

from sklearn.linear_model import LogisticRegression


baseline_model = LogisticRegression(random_state=100, class_weight='balanced') # `weight of class` balancing technique used
baseline_model = baseline_model.fit(X_train, y_train)

y_train_pred = baseline_model.predict_proba(X_train)[:,1]
y_test_pred  = baseline_model.predict_proba(X_test)[:,1]

In [ ]:
y_train_pred = pd.Series(y_train_pred,index = X_train.index, ) # converting test and train to a series to preserve index
y_test_pred = pd.Series(y_test_pred,index = X_test.index)

In [ ]:
# Function for Baseline Performance Metrics
import math
def model_metrics(matrix) :
    TN = matrix[0][0]
    TP = matrix[1][1]
    FP = matrix[0][1]
    FN = matrix[1][0]
    accuracy = round((TP + TN)/float(TP+TN+FP+FN),3)
    print('Accuracy :' ,accuracy )
    sensitivity = round(TP/float(FN + TP),3)
    print('Sensitivity / True Positive Rate / Recall :', sensitivity)
    specificity = round(TN/float(TN + FP),3)
    print('Specificity / True Negative Rate : ', specificity)
    precision = round(TP/float(TP + FP),3)
    print('Precision / Positive Predictive Value :', precision)
    print('F1-score :', round(2*precision*sensitivity/(precision + sensitivity),3))

In [ ]:
# Prediction at threshold of 0.5 
classification_threshold = 0.5 
    
y_train_pred_classified = y_train_pred.map(lambda x : 1 if x > classification_threshold else 0)
y_test_pred_classified = y_test_pred.map(lambda x : 1 if x > classification_threshold else 0)

In [ ]:
from sklearn.metrics import confusion_matrix
train_matrix = confusion_matrix(y_train, y_train_pred_classified)
print('Confusion Matrix for train:\n', train_matrix)
test_matrix = confusion_matrix(y_test, y_test_pred_classified)
print('\nConfusion Matrix for test: \n', test_matrix)

In [ ]:
# Baseline Model Performance : 

print('Train Performance : \n')
model_metrics(train_matrix)

print('\n\nTest Performance : \n')
model_metrics(test_matrix)

In [ ]:
# Specificity / Sensitivity Tradeoff 

# Classification at probability thresholds between 0 and 1 
y_train_pred_thres = pd.DataFrame(index=X_train.index)
thresholds = [float(x)/10 for x in range(10)]

def thresholder(x, thresh) :
    if x > thresh : 
        return 1 
    else : 
        return 0

    
for i in thresholds:
    y_train_pred_thres[i]= y_train_pred.map(lambda x : thresholder(x,i))
y_train_pred_thres.head()

In [ ]:
# # sensitivity, specificity, accuracy for each threshold
metrics_df = pd.DataFrame(columns=['sensitivity', 'specificity', 'accuracy'])

# Function for calculation of metrics for each threshold
def model_metrics_thres(matrix) :
    TN = matrix[0][0]
    TP = matrix[1][1]
    FP = matrix[0][1]
    FN = matrix[1][0]
    accuracy = round((TP + TN)/float(TP+TN+FP+FN),3)
    sensitivity = round(TP/float(FN + TP),3)
    specificity = round(TN/float(TN + FP),3)
    return sensitivity,specificity,accuracy

# generating a data frame for metrics for each threshold
for thres,column in zip(thresholds,y_train_pred_thres.columns.to_list()) : 
    confusion = confusion_matrix(y_train, y_train_pred_thres.loc[:,column])
    sensitivity,specificity,accuracy = model_metrics_thres(confusion)
    
    metrics_df =  metrics_df.append({ 
        'sensitivity' :sensitivity,
        'specificity' : specificity,
        'accuracy' : accuracy
    }, ignore_index = True)
    
metrics_df.index = thresholds
metrics_df

In [ ]:
metrics_df.plot(kind='line', figsize=(24,8), grid=True, xticks=np.arange(0,1,0.02),
                title='Specificity-Sensitivity TradeOff');

In [ ]:
optimum_cutoff = 0.45
y_train_pred_final = y_train_pred.map(lambda x : 1 if x > optimum_cutoff else 0)
y_test_pred_final = y_test_pred.map(lambda x : 1 if x > optimum_cutoff else 0)

train_matrix = confusion_matrix(y_train, y_train_pred_final)
print('Confusion Matrix for train:\n', train_matrix)
test_matrix = confusion_matrix(y_test, y_test_pred_final)
print('\nConfusion Matrix for test: \n', test_matrix)

In [ ]:
print('Train Performance: \n')
model_metrics(train_matrix)

print('\n\nTest Performance : \n')
model_metrics(test_matrix)

In [ ]:
# ROC_AUC score 
from sklearn.metrics import roc_auc_score
print('ROC AUC score for Train : ',round(roc_auc_score(y_train, y_train_pred),3), '\n' )
print('ROC AUC score for Test : ',round(roc_auc_score(y_test, y_test_pred),3) )

## Model is overfitting given we have a lot of variables present in it and its test-score is very less
## It has very high TPR which is overfitting in case of the test data 

#### Generializing model building to improve on the model building and reduce overfitting 

In [ ]:
from sklearn.feature_selection import RFE
#from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
rfe = RFE(estimator=lr,n_features_to_select=25)
results = rfe.fit(X_train,y_train)

# DataFrame with features supported by RFE
rfe_support = pd.DataFrame({'Column' : X.columns.to_list(), 'Rank' : rfe.ranking_, 
                                      'Support' :  rfe.support_}).sort_values(by=
                                       'Rank', ascending=True)
rfe_support

In [ ]:
# RFE Selected columns
rfe_selected_columns = rfe_support.loc[rfe_support['Rank'] == 1,'Column'].to_list()
rfe_selected_columns

In [ ]:
# Logistic Regression Model with RFE columns
import statsmodels.api as sm 

# Note that the SMOTE resampled Train set is used with statsmodels.api.GLM since it doesnot support class_weight
logr = sm.GLM(y_train_resampled,(sm.add_constant(X_train_resampled[rfe_selected_columns])), family = sm.families.Binomial())
logr_fit = logr.fit()
logr_fit.summary()

In [ ]:
# Using P-value and vif for manual feature elimination
from statsmodels.stats.outliers_influence import variance_inflation_factor
def vif(X_train_resampled, logr_fit, selected_columns) : 
    vif = pd.DataFrame()
    vif['Features'] = rfe_selected_columns
    vif['VIF'] = [variance_inflation_factor(X_train_resampled[selected_columns].values, i) for i in range(X_train_resampled[selected_columns].shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.set_index('Features')
    vif['P-value'] = round(logr_fit.pvalues,4)
    vif = vif.sort_values(by = ["VIF",'P-value'], ascending = [False,False])
    return vif

vif(X_train_resampled, logr_fit, rfe_selected_columns)

In [ ]:
## We have VIF of around 4, but there are few variables which have higher p-value , we can remove that variable

In [ ]:
## removing variable sachet_3g_7 based on p-values
selected_columns = rfe_selected_columns
selected_columns.remove('sachet_3g_7')
selected_columns

In [ ]:
## building 2nd model 
logr2 = sm.GLM(y_train_resampled,(sm.add_constant(X_train_resampled[selected_columns])), family = sm.families.Binomial())
logr2_fit = logr2.fit()
logr2_fit.summary()

In [ ]:
# vif and p-values
vif(X_train_resampled, logr2_fit, selected_columns)

In [ ]:
## removing std_ic_t2t_mou_7_perc based on p-value
selected_columns.remove('std_ic_t2t_mou_7_perc')
selected_columns

In [ ]:
## Building 3rd model 
logr3 = sm.GLM(y_train_resampled,(sm.add_constant(X_train_resampled[selected_columns])), family = sm.families.Binomial())
logr3_fit = logr3.fit()
logr3_fit.summary()

In [ ]:
# vif and p-values
vif(X_train_resampled, logr3_fit, selected_columns)

In [ ]:
## removing sachet_delta_2g based on p-value
selected_columns.remove('sachet_delta_2g')
selected_columns

In [ ]:
## Building 4th model 
logr4 = sm.GLM(y_train_resampled,(sm.add_constant(X_train_resampled[selected_columns])), family = sm.families.Binomial())
logr4_fit = logr4.fit()
logr4_fit.summary()

In [ ]:
# vif and p-values
vif(X_train_resampled, logr4_fit, selected_columns)

In [ ]:
## removing loc_ic_t2m_mou_8_perc based on VIF value 
selected_columns.remove('loc_ic_t2m_mou_8_perc')
## Building 5th model 
logr5 = sm.GLM(y_train_resampled,(sm.add_constant(X_train_resampled[selected_columns])), family = sm.families.Binomial())
logr5_fit = logr5.fit()
# vif and p-values
vif(X_train_resampled, logr5_fit, selected_columns)

In [ ]:
## All the model values are lesser than 4 in VIF and p-value in range of 0.05 so this will be our final model 
## to work with 
logr5_fit.summary()


In [ ]:
# Prediction 
y_train_pred_lr = logr5_fit.predict(sm.add_constant(X_train_resampled[selected_columns]))
y_train_pred_lr.head()

In [ ]:
y_test_pred_lr = logr5_fit.predict(sm.add_constant(X_test[selected_columns]))
y_test_pred_lr.head()

In [ ]:
## Performance
## finding the sensitivity and specificity with respect to the thresholds 
# Specificity / Sensitivity Tradeoff 

# Classification at probability thresholds between 0 and 1 
y_train_pred_thres = pd.DataFrame(index=X_train_resampled.index)
thresholds = [float(x)/10 for x in range(10)]

def thresholder(x, thresh) :
    if x > thresh : 
        return 1 
    else : 
        return 0

    
for i in thresholds:
    y_train_pred_thres[i]= y_train_pred_lr.map(lambda x : thresholder(x,i))
y_train_pred_thres.head()

In [ ]:
#DataFrame for Performance metrics at each threshold

logr_metrics_df = pd.DataFrame(columns=['sensitivity', 'specificity', 'accuracy'])
for thres,column in zip(thresholds,y_train_pred_thres.columns.to_list()) : 
    confusion = confusion_matrix(y_train_resampled, y_train_pred_thres.loc[:,column])
    sensitivity,specificity,accuracy = model_metrics_thres(confusion)
    logr_metrics_df =  logr_metrics_df.append({ 
        'sensitivity' :sensitivity,
        'specificity' : specificity,
        'accuracy' : accuracy
    }, ignore_index = True)
    
logr_metrics_df.index = thresholds
logr_metrics_df

## at 0.5 we have the best sensitivity and specificity and accuracy based on how close these values are 
## range from 0.4 to 0.5 is acting really greate place to start with model performance 

In [ ]:
logr_metrics_df.plot(kind='line', figsize=(24,8), grid=True, xticks=np.arange(0,1,0.02),
                title='Specificity-Sensitivity TradeOff');

## This  graph is very specifically crossing at around 0.5 as threshold, which should be the go to place 
#3 for the prediction

In [ ]:
optimum_cutoff = 0.497
y_train_pred_lr_final = y_train_pred_lr.map(lambda x : 1 if x > optimum_cutoff else 0)
y_test_pred_lr_final = y_test_pred_lr.map(lambda x : 1 if x > optimum_cutoff else 0)

train_matrix = confusion_matrix(y_train_resampled, y_train_pred_lr_final)
print('Confusion Matrix for train:\n', train_matrix)
test_matrix = confusion_matrix(y_test, y_test_pred_lr_final)
print('\nConfusion Matrix for test: \n', test_matrix)

In [ ]:
print('Train Performance: \n')
model_metrics(train_matrix)

print('\n\nTest Performance : \n')
model_metrics(test_matrix)

Now we have reduced the overfitting after removing the columns and also we can se how our model is predicting the classes 
correctly but it has very low precision, which might be that case tha our over-samling technique needs some tuning to make
it better

In [ ]:
# ROC_AUC score 
print('ROC AUC score for Train : ',round(roc_auc_score(y_train_resampled, y_train_pred_lr),3), '\n' )
print('ROC AUC score for Test : ',round(roc_auc_score(y_test, y_test_pred_lr),3) )

In [ ]:
## Interpretable model summary
lr_summary_html = logr5_fit.summary().tables[1].as_html()
lr_results = pd.read_html(lr_summary_html, header=0, index_col=0)[0]
coef_column = lr_results.columns[0]
print('Most important predictors of Churn , in order of importance and their coefficients are as follows : \n')
lr_results.sort_values(by=coef_column, key=lambda x: abs(x), ascending=False)['coef']

### Building Non-linear model using PCA and boosting algorithms 

In [ ]:
from sklearn.decomposition import PCA 
pca = PCA(random_state = 42) 
pca.fit(X_train_resampled) # note that pca is fit on resampled train set. 
pca.components_

In [ ]:
pca.explained_variance_ratio_

In [ ]:
## Scree Plot to understand variance by components
var_cum = np.cumsum(pca.explained_variance_ratio_)
plt.figure(figsize=(20,8))
sns.set_style('darkgrid')
sns.lineplot(np.arange(1,len(var_cum) + 1), var_cum)
plt.xticks(np.arange(0,140,5))
plt.axhline(0.8,color='g')
plt.axhline(0.95,color='r')
plt.axhline(1.0,color='r')
plt.axvline(25,color='b')
plt.axvline(50,color='b')
plt.axvline(100,color='b')
plt.text(10,0.81,'0.80')
plt.text(10,0.96,'0.95')

plt.title('Scree Plot of Telecom Churn Train Set');

Variables above 100 and less than 105 explains about 95% of the total variance in the data set, we can now go ahead with use of 105 components to build the model

In [ ]:
# Perform PCA using the first 55 components
pca_final = PCA(n_components=105, random_state=42)
transformed_data = pca_final.fit_transform(X_train_resampled)
X_train_pca = pd.DataFrame(transformed_data, columns=["PC_"+str(x) for x in range(1,106)], index = X_train_resampled.index)
data_train_pca = pd.concat([X_train_pca, y_train_resampled], axis=1)

data_train_pca.head()

In [ ]:
## Plotting principal components 
sns.pairplot(data=data_train_pca, x_vars=["PC_1"], y_vars=["PC_2"], hue = "churn", size=8);

In [ ]:
# X,y Split
y_train_pca = data_train_pca.pop('churn')
X_train_pca = data_train_pca

# Transforming test set with pca ( 45 components)
X_test_pca = pca_final.transform(X_test)

# Logistic Regression
lr_pca = LogisticRegression(random_state=100, class_weight='balanced')
lr_pca.fit(X_train_pca,y_train_pca ) 

In [ ]:
# y_train predictions
y_train_pred_lr_pca = lr_pca.predict(X_train_pca)
y_train_pred_lr_pca[:5]

In [ ]:
# Test Prediction
X_test_pca = pca_final.transform(X_test)
y_test_pred_lr_pca = lr_pca.predict(X_test_pca)
y_test_pred_lr_pca[:5]

In [ ]:
train_matrix = confusion_matrix(y_train_resampled, y_train_pred_lr_pca)
test_matrix = confusion_matrix(y_test, y_test_pred_lr_pca)

print('Train Performance :\n')
model_metrics(train_matrix)

print('\nTest Performance :\n')
model_metrics(test_matrix)

In [ ]:
# Creating a Logistic regression model using pca transformed train set
from sklearn.pipeline import Pipeline
lr_pca = LogisticRegression(random_state=100, class_weight='balanced')

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV , StratifiedKFold
params = {
    'penalty' : ['l1','l2','none'], 
    'C' : [0,1,2,3,4,5,10,50]
}
folds = StratifiedKFold(n_splits=4, shuffle=True, random_state=100)

search = GridSearchCV(cv=folds, estimator = lr_pca, param_grid=params,scoring='roc_auc', verbose=True)
search.fit(X_train_pca, y_train_pca)

In [ ]:
# Optimum Hyperparameters
print('Best ROC-AUC score :', search.best_score_)
print('Best Parameters :', search.best_params_)

In [ ]:
# Modelling using the best LR-PCA estimator 
lr_pca_best = search.best_estimator_
lr_pca_best_fit = lr_pca_best.fit(X_train_pca, y_train_pca)

# Prediction on Train set
y_train_pred_lr_pca_best = lr_pca_best_fit.predict(X_train_pca)
y_train_pred_lr_pca_best[:5]

In [ ]:
# Prediction on test set
y_test_pred_lr_pca_best = lr_pca_best_fit.predict(X_test_pca)
y_test_pred_lr_pca_best[:5]

In [ ]:
## Model Performance after Hyper Parameter Tuning

train_matrix = confusion_matrix(y_train_resampled, y_train_pred_lr_pca_best)
test_matrix = confusion_matrix(y_test, y_test_pred_lr_pca_best)
print(test_matrix)

print('Train Performance :\n')
model_metrics(train_matrix)

print('\nTest Performance :\n')
model_metrics(test_matrix)

As it was expected, linear model with PCA will fail to work better , given we are overfitting the data and also due to orthogonal transformation of the variables , linear function fails to work efficiently, we can try with some tree models and we can use boosting or bagging to improve the fitting of the models

### PCA with Random Forest

In [ ]:
y_train_resampled.value_counts()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# creating a random forest classifier using pca output

pca_rf = RandomForestClassifier(random_state=42, class_weight= {0 :0.5 , 1 : 0.5 } , oob_score=True, n_jobs=-1,verbose=1)
pca_rf

In [ ]:
# Hyper parameter Tuning
params = {
    'n_estimators'  : [50,100],
    'max_depth' : [5,6,7],
    'min_samples_leaf' : [25,30]
}
folds = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
pca_rf_model_search = GridSearchCV(estimator=pca_rf, param_grid=params, 
                                   cv=folds, scoring='f1', verbose=False)

pca_rf_model_search.fit(X_train_pca, y_train_resampled)

In [ ]:
# Optimum Hyperparameters
print('Best F1 score :', pca_rf_model_search.best_score_)
print('Best Parameters :', pca_rf_model_search.best_params_)

In [ ]:
pca_rf_model_search.cv_results_

In [ ]:
# Modelling using the best PCA-RandomForest Estimator 
pca_rf_best = pca_rf_model_search.best_estimator_
pca_rf_best_fit = pca_rf_best.fit(X_train_pca, y_train_resampled)

# Prediction on Train set
y_train_pred_pca_rf_best = pca_rf_best_fit.predict(X_train_pca)
y_train_pred_pca_rf_best[:5]

In [ ]:
# Prediction on test set
y_test_pred_pca_rf_best = pca_rf_best_fit.predict(X_test_pca)
y_test_pred_pca_rf_best[:5]

In [ ]:
## PCA - RandomForest Model Performance - Hyper Parameter Tuned

train_matrix = confusion_matrix(y_train_resampled, y_train_pred_pca_rf_best)
test_matrix = confusion_matrix(y_test, y_test_pred_pca_rf_best)

print('Train Performance :\n')
model_metrics(train_matrix)

print('\nTest Performance :\n')
model_metrics(test_matrix)

In [ ]:
## out of bag error 
pca_rf_best_fit.oob_score_

In [ ]:
## Recommendations
print('Most Important Predictors of churn , in the order of importance are : ')
lr_results.sort_values(by=coef_column, key=lambda x: abs(x), ascending=False)['coef']

From the above, the following are the strongest indicators of churn are:


- Roaming customers have higher chance to churn, providing roaming prices might not be as efficient. Ones opting out of roaming has better chance to not churn
- Increase in incoming minutes of usage in the action phase can reduce churn rate.
- Customers who churn show lower average monthly local incoming calls from fixed line in the action period, local outgoing calls to mobile operators and local incoming from same operator gives lesser churn.
- Customers who churn show lower number of recharges done in action period by .94 standard deviations, when all other factors are held constant. This is the second strongest indicator of churn.
- Further customers who churn have done 0.6 standard deviations higher recharge than non-churn customers in good phase. This factor when coupled with above factors is a good indicator of churn.
- STD & ISD calling has resulted in increase of churn, these facilities might have price points which are not attractive for the users
- Other services result in the higher churning, these after/other services need an important input to be checked and maintained


#### Model Selection
- Models with high sensitivity are the best for predicting churn. Use the PCA + Logistic Regression model to predict churn. It has an ROC score of 0.93, test sensitivity of 75.9%